In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## import keras-tensor flow packages
#A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow

In [6]:
df=pd.read_csv('Ecommerce Customers.csv')

In [7]:
df.head()

,Email,Address,Avatar,Avg. Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,mstephenson@fernandez.com,"835 Frank Tunnel\nWrightmouth, MI 82180-9605",Violet,34.497268,12.655651,39.577668,4.082621,587.951054
1,hduke@hotmail.com,"4547 Archer Common\nDiazchester, CA 06566-8576",DarkGreen,31.926272,11.109461,37.268959,2.664034,392.204933
2,pallen@yahoo.com,"24645 Valerie Unions Suite 582\nCobbborough, D...",Bisque,33.000915,11.330278,37.110597,4.104543,487.547505
3,riverarebecca@gmail.com,"1414 David Throughway\nPort Jason, OH 22070-1220",SaddleBrown,34.305557,13.717514,36.721283,3.120179,581.852344
4,mstephens@davidson-herman.com,"14023 Rodriguez Passage\nPort Jacobville, PR 3...",MediumAquaMarine,33.330673,12.795189,37.536653,4.446308,599.406092


In [34]:
df.shape

(500, 8)

In [8]:
X = df[['Avg. Session Length', 'Time on App','Length of Membership','Time on Website']]
y = df[['Yearly Amount Spent']]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Normalize the Data

In [13]:
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()

X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test) 


In [15]:
# find the number of input features
n_features = X_train.shape[1]
print(n_features)

4


## Define the model

In [32]:
# define the model

# This indicates that the model will be a linear stack of layers. You can add layers to it one by one.
model = Sequential()

# Input layer with 10 neurons
model.add(Dense(10, kernel_initializer = 'normal', activation = 'relu',input_shape = (4,))) 

# Hidden layer with 8 neurons
model.add(Dense(8, activation='relu'))

 # Output layer for linear regression
model.add(Dense(1))

# Compile the model

In [33]:
from tensorflow.keras.optimizers import RMSprop
model.compile(
   optimizer = RMSprop(), 
   loss = 'mse',   
   metrics = ['mean_absolute_error']
)

## Train(Fit) the model

In [37]:
model.fit(
   X_train, y_train,    
   batch_size=50, 
   epochs = 20, 
   verbose = 1, 
   validation_split = 0.2, 
)


Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 247669.5625 - mean_absolute_error: 490.2487 - val_loss: 256695.5000 - val_mean_absolute_error: 500.9287
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 247253.3906 - mean_absolute_error: 489.8369 - val_loss: 256253.8438 - val_mean_absolute_error: 500.5026
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 246807.6875 - mean_absolute_error: 489.3961 - val_loss: 255781.7812 - val_mean_absolute_error: 500.0471
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 246331.3906 - mean_absolute_error: 488.9250 - val_loss: 255277.8438 - val_mean_absolute_error: 499.5609
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 245823.3438 - mean_absolute_error: 488.4225 - val_loss: 254741.0469 - val_mean_absolute_error: 499.0429
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 245283.0469 - mean_absolute_error: 487.8881 - val_loss: 254171.2969 - val_mean_absolute_error: 498.4928
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━

## Evaluate the model

In [38]:
model.evaluate(X_test, y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 224810.5469 - mean_absolute_error: 469.4843 


[227358.125, 472.0164489746094]

## Prediction

In [39]:

predictions = model.predict(X_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [41]:
from sklearn import metrics


In [42]:
metrics.mean_absolute_error(y_test,predictions)

472.0164530309631

# Hyperparameter Tuning

## parameter to be tuned
- Learning Rate
- Batch Size

In [57]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(1, input_shape=(4,), kernel_initializer='normal'))
    
    # Tune the optimizer
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])
    
    # Tune the learning rate for the optimizer
    if hp_optimizer == 'adam':
        hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
    else:
        hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        optimizer = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model


In [58]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='linear_regression_tuning'
)


Reloading Tuner from my_dir\linear_regression_tuning\tuner0.json


In [59]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2)


Trial 10 Complete [00h 00m 10s]
val_loss: 259439.75

Best val_loss So Far: 239.2466278076172
Total elapsed time: 00h 20m 00s


In [60]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)



C:\Users\ngugi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
# Train the best model
best_model.fit(X_train, y_train, epochs=100, validation_split=0.2)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 239011.5156 - val_loss: 221753.8750
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 203396.5625 - val_loss: 189210.5000
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 173099.3125 - val_loss: 161464.5938
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 147324.3281 - val_loss: 137806.7969
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 125395.4219 - val_loss: 117632.9141
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 106737.6484 - val_loss: 100428.2109
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 90862.1797 - val_loss: 85754.3047
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 77353.4141 - val_loss: 73237.6094
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 65857.9297 - val_loss: 62559.9141
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 56075.1719 - val_loss: 53450.0000
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - l

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 97.6163 - val_loss: 120.7139
Epoch 67/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 96.6303 - val_loss: 118.6316
Epoch 68/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 95.7955 - val_loss: 116.7835
Epoch 69/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 95.0893 - val_loss: 115.1406
Epoch 70/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 94.4922 - val_loss: 113.6778
Epoch 71/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 93.9878 - val_loss: 112.3740
Epoch 72/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 93.5620 - val_loss: 111.2090
Epoch 73/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 93.2028 - val_loss: 110.1668
Epoch 74/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 92.9003 - val_loss: 109.2337
Epoch 75/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 92.6458 - val_loss: 108.3967
Epoch 76/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 92.4321 - val_loss: 107.6449
Epoch 77/100
10/10 ━━━━━━━━━━━━

In [64]:
best_model.evaluate(X_test, y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 100.0813


110.78744506835938

In [65]:
y_pred = best_model.predict(X_test)

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/stepWARNING:tensorflow:6 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001C55EE9E170> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [66]:
metrics.mean_absolute_error(y_test,y_pred)

8.553041635035159